In [ ]:
import pandas as pd
import requests
from splinter import Browser

In [ ]:
executable_path = {'executable_path': '/bin/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless = True)

In [ ]:
drafts = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]
all_data =pd.DataFrame()
for year in drafts:
    url = f'https://www.perfectgame.org/Draft/Signings.aspx?y={year}'
    browser.visit(url)
    year_data = pd.DataFrame()
    for d in range(40):
        round_num = d+1
        roundselector = browser.find_by_id('ContentPlaceHolder1_ddlRound').first
        roundselector.select_by_text(f'{round_num}')
        draft = pd.read_html(browser.html)
        drafts = draft[4]
        drafts['Year']= year
        year_data = pd.concat([year_data, drafts])
    all_data = pd.concat([all_data, year_data])
    
browser.quit()

In [ ]:
all_data.to_csv("Dataset/draft_data.csv")

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float

In [ ]:
base = declarative_base()

In [ ]:
class Drafts(base):
    __tablename__ = 'drafts'
    Id = Column(Integer, primary_key =True)
    Rd = Column(Integer)
    Pick = Column(Integer)
    Team = Column(String(3))
    Player = Column(String(200))
    Pos = Column(String(3))
    HS = Column(String(200))
    St = Column(String(2))
    Signed = Column(String(10))
    Bonus = Column(Integer)
    Year =  Column(Integer)
    

In [ ]:
engine =  create_engine("sqlite:///Dataset/drafts.sqlite")
conn = engine.connect()

In [ ]:
Base.metadata.drop_all(engine)
base.metadata.create_all(engine)

In [ ]:
draft_data = pd.read_csv("Dataset/draft_data.csv")
draft_data.head()

In [ ]:
draft_data = draft_data.drop(columns =["Unnamed: 0"])
draft_data.head()

In [ ]:
from sqlalchemy.orm import Session

In [ ]:
session = Session(bind = engine)

In [ ]:
draft_data.to_sql(name = 'drafts', con = engine, if_exists ="append", index =False)

In [ ]:
picks = session.query(Drafts)

In [ ]:
for i in picks:
    print(i.Year)